## load and clean student data and their prefences

In [11]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment


file_location = r"C:\Users\g21728\Desktop\Masterthesis_SS22.xlsx"
# file_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Bachelor-Thesis.xlsx"
# file_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Masterseminar.xlsx"
# file_location = r"C:\Users\g21728\Desktop\WS21\Übersicht_Teilnehmer_Proseminar.xlsx"

df = pd.read_excel(file_location, sheet_name="Tabelle1")

#optional
#get student grades to order list descending
df_grades = pd.read_excel(r"C:\Users\g21728\Desktop\Zuteilung_Wöhrmann_wissenschaftliche Arbeiten_SoSe22.xlsx")
df_grades.rename(columns={"matrikelnummer": "Matrnr"}, inplace=True)
df_grades = df_grades[["Matrnr", "Durchschnittsnote"]]
df_grades[:3]
#merge with grades and sort descending
df = df.merge(df_grades, how="left", on="Matrnr")
df.sort_values(by=["Durchschnittsnote"], ascending=False, inplace=True,)
df
#

#rename Themenpräferenz column
df.rename(columns={"Themenpräferenz": "Themen", "Themen-Präferenzen": "Themen"}, inplace=True)

# drop zurückgetreten students
df = df[df["Themen"].str.contains("zurückgetreten|Rücktritt|abgemeldet", regex=True)==False]

# convert float columns to int
# df.dtypes # datatype of columns
df["Matrnr"] = df["Matrnr"].astype("Int64") #Int64 can store missing values as NaN

# remove whitespaces from lists
df["Themen"] = df["Themen"].str.replace(' ', '') 
df["Themen"] = df["Themen"].apply(eval) # lists are stored as strings in pandas -> convert with apply(eval)

number_of_topics = len(df.iloc[1,5])
print(f"Number of different topics: {number_of_topics}")

#############
#if one topic is assigned multiple times, then multiple each topic n times (in seminar courses):
if "seminar" in file_location.lower():
    topic_multiplier = [4,4,3] #4,4,4 -> each topic is offered 4 times
    df["Themen"] = df["Themen"].apply(lambda x: np.repeat(x, topic_multiplier))
    
    total_topics = sum(topic_multiplier)
    print(f"Total number of topics: {total_topics}")
#############

print(f"Number of students: {df.shape[0]}") #number of rows

df.reset_index(inplace=True)
df.drop(columns=["index"], inplace=True)
df[:2]

Number of different topics: 6
Number of students: 6


,Pos.,Matrnr,Name,Vorname,Bearbeitungszeitraum,Themen,Thema DE,Thema ENG,Betreuer,Note,E-Mail,Studiengang,Bemerkungen,Durchschnittsnote
0,2.0,1029691,Dany,Ricarda,27.04.22 - 24.08.22,"[2, 1, 3, 4, 5, 6]",NaN,NaN,NaN,NaN,ricarda.dany@wirtschaft.uni-giessen.de,BWL alt 60 CP,60-CP Master,12.33
1,4.0,3045189,König,Fabio,28.04.22 - 24.10.22,"[4, 6, 1, 2, 5, 3]",NaN,NaN,NaN,NaN,fabio.koenig@umwelt.uni-giessen.de,MSc BWL neu (Studienbeginn WS 18/19 oder später),Business Administration,12.33


## assign topics

In [12]:
# preferences = [
#     [4,3,1,2], 
#     [2,4,1,3], 
#     [2,1,3,4], 
#     ]

preferences = df['Themen'].to_list()

cost = np.array(preferences)
row_ind, col_ind = linear_sum_assignment(cost) #results stored as list in col_ind
adj_ind = [i+1 for i in col_ind] #adjust by plus one since it starts at zero

#############
if "seminar" in file_location.lower():
    cumsum = np.cumsum(topic_multiplier) #cumulative sum of the elements

    #if more students than topics, then adj_ind is misleading. check in which topic range adj_ind is assigned: 
    new_list = []
    for i in adj_ind:
        for index,value in enumerate(cumsum):
            if i <= value and index == 0:
                new_list.append(index+1)
            elif i <= value and i > cumsum[index-1]:
                new_list.append(index+1)
    adj_ind = new_list #overwrite old list
#############


print(f"Available Topics: {len(preferences[0])}")
print(f"Assigned Topics: {len(col_ind)}")
print(f"Number of students: {df.shape[0]}")
print(f"array indices of best solution: {col_ind}")
print(f"adjusted (plus one) indices of best solution: {adj_ind}\n")


for i,v in enumerate(preferences):
    selected_topic = adj_ind[i]
    pref_of_selected_topic = v[col_ind[i]]
    print(f"Student {i+1}: Assigned topic {selected_topic} (selected as #{pref_of_selected_topic})")
print(f"\nTotal Sum of preferences (lowest sum possible): {cost[row_ind, col_ind].sum()}\n")

df["assigned_topic"] = adj_ind # copy assigned topics to column

#move columns
col = df.pop("assigned_topic")
df.insert(df.columns.get_loc("Themen") + 1, col.name, col)

#append new sheet to existing excel file
with pd.ExcelWriter(file_location, mode='a', engine='openpyxl') as writer:  
    df.to_excel(writer, sheet_name='assigned', index=False)

Available Topics: 6
Assigned Topics: 6
Number of students: 6
array indices of best solution: [1 2 0 4 3 5]
adjusted (plus one) indices of best solution: [2, 3, 1, 5, 4, 6]

Student 1: Assigned topic 2 (selected as #1)
Student 2: Assigned topic 3 (selected as #1)
Student 3: Assigned topic 1 (selected as #2)
Student 4: Assigned topic 5 (selected as #1)
Student 5: Assigned topic 4 (selected as #1)
Student 6: Assigned topic 6 (selected as #4)

Total Sum of preferences (lowest sum possible): 10



### merge assigned topics with Betreuer sheet

In [98]:
df2 = pd.read_excel(file_location, sheet_name="Themen")
# df2

df3 = df.merge(df2, how="left", left_on="scipy.optimize.linear_sum_assignment", right_on="Thema")
df3["Betreuer_x"] = df3["Betreuer_y"]
df3.rename(columns={"Betreuer_x":"Betreuer"}, inplace=True)
df3["Thema DE"] = df3["Titel DE"]
df3.drop(["Titel DE", "Thema", "Betreuer_y"],1, inplace=True)

# df3
df3.to_excel(save_location, sheet_name="Final", index=False)

### adjust weights of specific students

In [ ]:
prefe = df['Themen-Präferenzen'].to_list() # can be directly modified by accessing prefe

# prefer X first students by overweighting their first coice
preferred_number = 1
# prefer first students by adding X to each other student -> doesnt work
# for i in range(len(prefe)):
#     if i >= preferred_number:
#         x = list(map(lambda x:x+1, prefe[i])) # add 1 to each preference weight for students after preferred students
#         df.loc[i, "Themen-Präferenzen"] = str(x)
#     else:
#         df.loc[i, "Themen-Präferenzen"] = str(prefe[i])

# first choice of preferred students gets -1 weight -> more likely that preferred students get their first choice
# for i in range(len(prefe)):
#     if i < preferred_number:
#         index_min_value = prefe[i].index(min(prefe[i]))
#         prefe[i][index_min_value] -= 1 # adjust weight of first choice by minus one
#         print(prefe[i])
#         df.loc[i, "Themen-Präferenzen"] = str(prefe[i])
#     else:
#         df.loc[i, "Themen-Präferenzen"] = str(prefe[i])
# df["Themen-Präferenzen"] = df["Themen-Präferenzen"].apply(eval) # lists are stored as strings in pandas -> convert with apply(eval)

# ignore a specific topic of all students (otherwise unpopular topic gets ignored if more topics avaiable than students)
for i in range(len(prefe)):
    # print(prefe[i][5:8])
    prefe[i].pop() # drop last element. directly modifies dataframe
    prefe[i][5] = 99 # ignore topic at index X (5 -> drop topic 6)
    # prefe[i].pop(5) # drop 


# preferences = df['Themen-Präferenzen'].to_list()
# print(preferences)
df